In [1]:
import torch
from LabData.DataLoaders.PRSLoader import PRSLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from genetics_dataset import GeneticsDataset

data = GeneticsDataset()

Mapping files: 100%|██████████████████████████████| 3/3 [00:01<00:00,  2.09it/s]


In [2]:
class fc_snpnet(nn.Module):
    def __init__(self):
        super(fc_snpnet, self).__init__()
        self.conv1 = nn.Conv1d(31, 31, 2000)
        self.conv2 = nn.Conv1d(31, 31, 1000)
        self.conv4 = nn.Conv1d(31, 31, 1000)
        self.conv6 = nn.Conv1d(31, 31, 1000)
        self.conv7 = nn.Conv1d(31, 31, 1000)
        self.fc1 = nn.Linear(3596, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 516)
        self.fc4 = nn.Linear(516, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.reshape(x, [2, 31, 6053])
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv4(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = x.flatten()
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        x = self.fc6(x)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
net = fc_snpnet()
net = net.to(device)

In [3]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.2)
for epoch in range(30):
    i = 0
    for tenK_index in list(data.binaries.sample.values)[0:4000]:
        try:
            snps, label = data.__getitem__(tenK_index)
            snps = snps.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            outputs = net(snps)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            if i % 10 == 0: ##print every 50 people
                print("Epoch: ", epoch)
                print("Loss: ", loss)
                print("output: ", outputs, ", true height: ", label)
        except IndexError:
            pass
        i += 1

/home/zacharyl/miniconda3/envs/dl/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/zacharyl/miniconda3/envs/dl/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch:  0
Loss:  tensor(29071.6387, grad_fn=<MseLossBackward0>)
output:  tensor([-0.0041], grad_fn=<AddBackward0>) , true height:  tensor(170.5000)
Epoch:  0
Loss:  tensor(24.4964, grad_fn=<MseLossBackward0>)
output:  tensor([164.8506], grad_fn=<AddBackward0>) , true height:  tensor(169.8000)
Epoch:  0
Loss:  tensor(38.5752, grad_fn=<MseLossBackward0>)
output:  tensor([169.5891], grad_fn=<AddBackward0>) , true height:  tensor(175.8000)


KeyboardInterrupt: 